In [3]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from sklearn import linear_model
import csv
from statistics import median

import surprise
from surprise import SVD, Reader, Dataset
from surprise.model_selection import train_test_split

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [5]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, classification_report

In [6]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\antho\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
user_and_item_data71mb = []

f = gzip.open("user_and_item_data71mb.json.gz")
for l in f:
    d = eval(l)
    user_and_item_data71mb.append(d)
    if len(user_and_item_data71mb) >= 80000:
        break
        
f.close()

In [8]:
review_data_6mb = []

f = gzip.open("review_data_6mb.json.gz")
for l in f:
    d = eval(l)
    review_data_6mb.append(d)
    if len(review_data_6mb) >= 50000:
        break
        
f.close()

In [9]:
review_data_6mb[4]

{'user_id': 'maplemage',
 'user_url': 'http://steamcommunity.com/id/maplemage',
 'reviews': [{'funny': '3 people found this review funny',
   'posted': 'Posted April 15, 2014.',
   'last_edited': '',
   'item_id': '211420',
   'helpful': '35 of 43 people (81%) found this review helpful',
   'recommend': True,
   'review': 'Git gud'},
  {'funny': '1 person found this review funny',
   'posted': 'Posted December 23, 2013.',
   'last_edited': '',
   'item_id': '211820',
   'helpful': '12 of 16 people (75%) found this review helpful',
   'recommend': True,
   'review': "It's like Terraria, you play for 9 hours straight, get endgame armour then stop playing until the next update."},
  {'funny': '2 people found this review funny',
   'posted': 'Posted March 14, 2014.',
   'last_edited': '',
   'item_id': '730',
   'helpful': '5 of 5 people (100%) found this review helpful',
   'recommend': True,
   'review': 'Hold shift to win, Hold CTRL to lose.'},
  {'funny': '',
   'posted': 'Posted July 

In [10]:
user_and_item_data71mb[5]

{'user_id': 'MinxIsBetterThanPotatoes',
 'items_count': 371,
 'steam_id': '76561198004744620',
 'user_url': 'http://steamcommunity.com/id/MinxIsBetterThanPotatoes',
 'items': [{'item_id': '50',
   'item_name': 'Half-Life: Opposing Force',
   'playtime_forever': 256,
   'playtime_2weeks': 0},
  {'item_id': '240',
   'item_name': 'Counter-Strike: Source',
   'playtime_forever': 167,
   'playtime_2weeks': 0},
  {'item_id': '320',
   'item_name': 'Half-Life 2: Deathmatch',
   'playtime_forever': 3674,
   'playtime_2weeks': 0},
  {'item_id': '4000',
   'item_name': "Garry's Mod",
   'playtime_forever': 358,
   'playtime_2weeks': 0},
  {'item_id': '6570',
   'item_name': 'Onimusha 3: Demon Siege',
   'playtime_forever': 0,
   'playtime_2weeks': 0},
  {'item_id': '220',
   'item_name': 'Half-Life 2',
   'playtime_forever': 1299,
   'playtime_2weeks': 0},
  {'item_id': '340',
   'item_name': 'Half-Life 2: Lost Coast',
   'playtime_forever': 29,
   'playtime_2weeks': 0},
  {'item_id': '360',
  

In [11]:
len(user_and_item_data71mb)

80000

In [12]:
len(review_data_6mb)

25799

In [13]:
item_metadata = []

f = gzip.open("item_metadata.json.gz")
for l in f:
    d = eval(l)
    item_metadata.append(d)
    if len(item_metadata) >= 35000:
        break
        
f.close()

In [14]:
len(item_metadata)

32135

In [15]:
def merge_datasets_with_matching(review_data, user_and_item_data):
    """
    Merge datasets by matching user IDs with extensive debugging
    
    :param review_data: List of user review data
    :param user_and_item_data: List of user item and playtime data
    :return: List of merged data entries
    """
    # Verify input dataset lengths
    print(f"Original Review Data Length: {len(review_data)}")
    print(f"Original Items Data Length: {len(user_and_item_data)}")
    
    # Create lookup dictionaries for efficient matching
    review_dict = {entry['user_id']: entry for entry in review_data}
    items_dict = {entry['user_id']: entry for entry in user_and_item_data}
    
    # Detailed user ID analysis
    review_user_ids = set(review_dict.keys())
    items_user_ids = set(items_dict.keys())
    
    print(f"Unique User IDs in Review Data: {len(review_user_ids)}")
    print(f"Unique User IDs in Items Data: {len(items_user_ids)}")
    
    # Find common user IDs
    common_user_ids = review_user_ids & items_user_ids
    
    # More detailed tracking
    unmatched_review_users = review_user_ids - items_user_ids
    unmatched_items_users = items_user_ids - review_user_ids
    
    print(f"Common User IDs: {len(common_user_ids)}")
    print(f"Unmatched Users in Review Data: {len(unmatched_review_users)}")
    print(f"Unmatched Users in Items Data: {len(unmatched_items_users)}")
    
    # Sample of unmatched user IDs for investigation
    print("\nSample of Unmatched Review User IDs:")
    print(list(unmatched_review_users)[:10])
    print("\nSample of Unmatched Items User IDs:")
    print(list(unmatched_items_users)[:10])
    
    # Store merged data
    merged_data = []
    
    # Tracking variables
    total_reviews_processed = 0
    total_merged_entries = 0
    
    # Process only users present in both datasets
    for user_id in common_user_ids:
        user_reviews = review_dict[user_id]['reviews']
        user_items = items_dict[user_id]['items']
        
        total_reviews_processed += len(user_reviews)
        
        # Match reviews with items
        for review in user_reviews:
            # Find matching items
            matching_items = [
                item for item in user_items 
                if str(item['item_id']) == str(review['item_id'])
            ]
            
            # Create merged entries for matching items
            for item in matching_items:
                merged_entry = {
                    'user_id': user_id,
                    'item_id': review['item_id'],
                    'review_text': review.get('review', ''),
                    'recommend': review.get('recommend'),
                    'playtime_forever': item.get('playtime_forever', 0),
                    'playtime_2weeks': item.get('playtime_2weeks', 0)
                }
                merged_data.append(merged_entry)
                total_merged_entries += 1
    
    # Comprehensive summary
    print("\nFinal Matching Summary:")
    print(f"Total Users in Review Data: {len(review_dict)}")
    print(f"Total Users in Items Data: {len(items_dict)}")
    print(f"Common Users: {len(common_user_ids)}")
    print(f"Total Reviews Processed: {total_reviews_processed}")
    print(f"Total Merged Entries: {total_merged_entries}")
    
    return merged_data

In [16]:
merged_data = merge_datasets_with_matching(review_data_6mb,user_and_item_data71mb)

Original Review Data Length: 25799
Original Items Data Length: 80000
Unique User IDs in Review Data: 25485
Unique User IDs in Items Data: 79316
Common User IDs: 23925
Unmatched Users in Review Data: 1560
Unmatched Users in Items Data: 55391

Sample of Unmatched Review User IDs:
['GuyInTheGutter', '76561198093568871', 'redarmyjj', '76561198125761464', '76561198095653552', '76561198096025044', '76561198096565807', '76561198093043200', 'Azzaaaa', 'Nitoxis']

Sample of Unmatched Items User IDs:
['76561198068126490', '76561198074595218', '76561197988506370', '76561198070721183', '76561198048247974', 'tsunamaki', '76561198075687004', '76561198067648599', '76561198074673097', '76561198081948874']

Final Matching Summary:
Total Users in Review Data: 25485
Total Users in Items Data: 79316
Common Users: 23925
Total Reviews Processed: 55413
Total Merged Entries: 42210


In [17]:
merged_data[8]

{'user_id': '76561198060808593',
 'item_id': '4000',
 'review_text': 'Great Game!',
 'recommend': True,
 'playtime_forever': 377,
 'playtime_2weeks': 0}

In [18]:
len(merged_data)

42210

In [19]:
unique_user_ids = len(set(entry['user_id'] for entry in merged_data))
print(f"Number of unique user IDs: {unique_user_ids}")

# To get number of unique item IDs
unique_item_ids = len(set(entry['item_id'] for entry in merged_data))
print(f"Number of unique item IDs: {unique_item_ids}")

# If you want to see both at once
print("Unique user IDs:", len(set(entry['user_id'] for entry in merged_data)))
print("Unique item IDs:", len(set(entry['item_id'] for entry in merged_data)))

Number of unique user IDs: 18866
Number of unique item IDs: 3054
Unique user IDs: 18866
Unique item IDs: 3054


In [20]:
# Count recommendations
recommendations = [entry['recommend'] for entry in merged_data]

# Using numpy for a quick count
import numpy as np
unique, counts = np.unique(recommendations, return_counts=True)
print("Recommendation Distribution:")
print(dict(zip(unique, counts)))

# Alternatively, using a simple count method
true_count = recommendations.count(True)
false_count = recommendations.count(False)

print("\nRecommend True Count:", true_count)
print("Recommend False Count:", false_count)
print("Percentage of Recommendations:", true_count / len(recommendations) * 100, "%")

Recommendation Distribution:
{False: 4753, True: 37457}

Recommend True Count: 37457
Recommend False Count: 4753
Percentage of Recommendations: 88.7396351575456 %


In [21]:
import random

# Separate recommended and non-recommended entries
recommend_true = [entry for entry in merged_data if entry['recommend'] == True]
recommend_false = [entry for entry in merged_data if entry['recommend'] == False]

# Calculate target number of True recommendations (about 66%)
total_desired = len(recommend_false) * 1  # keep all False entries, add same number of True

# Randomly sample from True recommendations
balanced_recommend_true = random.sample(recommend_true, total_desired)

# Recombine the datasets
balanced_merged_data = recommend_false + balanced_recommend_true

# Shuffle the balanced dataset
random.shuffle(balanced_merged_data)

# Verify new distribution
recommendations = [entry['recommend'] for entry in balanced_merged_data]
unique, counts = np.unique(recommendations, return_counts=True)
print("New Recommendation Distribution:")
print(dict(zip(unique, counts)))

New Recommendation Distribution:
{False: 4753, True: 4753}


In [22]:
balanced_merged_data[0]

{'user_id': 'Kitakats',
 'item_id': '244850',
 'review_text': 'Annoying memory leaks, whichs causes long long lag spikes less than 1 fps which last for a few minutes at a time.',
 'recommend': False,
 'playtime_forever': 325,
 'playtime_2weeks': 0}

In [23]:
len(balanced_merged_data)

9506

In [24]:
# Ensure valid entries in balanced_merged_data and item_metadata
balanced_merged_data = [entry for entry in balanced_merged_data if 'item_id' in entry]
item_metadata = [entry for entry in item_metadata if 'id' in entry]

# Extract all item_ids from balanced_merged_data into a set for fast lookup
valid_item_ids = {entry['item_id'] for entry in balanced_merged_data}

# Filter the item_metadata list based on the presence of the 'id' in valid_item_ids
cleaned_item_metadata = [entry for entry in item_metadata if entry['id'] in valid_item_ids]

# Print the cleaned list size (optional)
print(f"Cleaned item_metadata contains {len(cleaned_item_metadata)} entries.")

Cleaned item_metadata contains 1558 entries.


In [25]:
# Extract all valid item names
valid_item_names = {
    item['item_name']
    for user in user_and_item_data71mb
    for item in user['items']
}

# Filter item_metadata with .get() for safe key access
cleaned_item_metadata = [
    entry for entry in item_metadata
    if entry.get('app_name') in valid_item_names or entry.get('title') in valid_item_names
]

# Print results
print(f"Cleaned item_metadata contains {len(cleaned_item_metadata)} entries.")

Cleaned item_metadata contains 7846 entries.


In [26]:
# Step 1: Extract all valid item_ids from balanced_merged_data into a set
valid_item_ids = {entry['item_id'] for entry in merged_data}

# Step 2: Filter cleaned_item_metadata based on presence of 'id' in valid_item_ids
cleaned_item_metadata_further = [
    entry for entry in cleaned_item_metadata
    if entry['id'] in valid_item_ids  # Direct comparison since both are strings
]

# Step 3: Print results
print(f"Further cleaned item_metadata contains {len(cleaned_item_metadata_further)} entries.")

Further cleaned item_metadata contains 2163 entries.


In [27]:
# Step 1: Extract all valid item_ids from balanced_merged_data into a set
valid_item_ids = {entry['item_id'] for entry in balanced_merged_data}

# Step 2: Filter cleaned_item_metadata based on presence of 'id' in valid_item_ids
cleaned_item_metadata_further = [
    entry for entry in cleaned_item_metadata
    if entry['id'] in valid_item_ids  # Direct comparison since both are strings
]

# Step 3: Print results
print(f"Further cleaned item_metadata contains {len(cleaned_item_metadata_further)} entries.")

Further cleaned item_metadata contains 1235 entries.


In [28]:
# Step 1: Search for the game with id '4000'
game_name = None

for entry in cleaned_item_metadata_further:
    if entry['id'] == '4000':  # Match the id
        game_name = entry['app_name']  # or entry['title'] depending on your preference
        break  # Exit loop once the game is found

# Step 2: Print the result
if game_name:
    print(f"Game name for id '4000': {game_name}")
else:
    print("Game with id '4000' not found.")

Game name for id '4000': Garry's Mod


In [29]:
X = [[entry['playtime_forever'] + entry['playtime_forever'],len(entry['review_text']),  # number of question marks
      1]  # bias term (constant feature)
     for entry in balanced_merged_data]

y = [entry['recommend'] for entry in balanced_merged_data]

In [30]:
len(X)

9506

### setting len(recommend_false) * 1 and no weght_class balanced gives slighly better performance

In [31]:
# Convert to numpy arrays
X = np.array(X)
y = np.array(y)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,shuffle=True)

# Train logistic regression
mod = LogisticRegression(C=1)
mod.fit(X_train, y_train)

# Evaluate
#print("Model Score:", mod.score(X_test, y_test))
#print("Coefficients:", mod.coef_)
predictions = mod.predict(X_test) # Binary vector of predictions
correct = predictions == y_test
print(sum(correct) / len(correct))

0.6020193521245267


In [32]:
# Random Baseline Classifier
baseline = DummyClassifier(strategy='stratified', random_state=42)
baseline.fit(X_train, y_train)
baseline_pred = baseline.predict(X_test)

print("Baseline (Random) Accuracy:", accuracy_score(y_test, baseline_pred))
print("\nBaseline Classification Report:")
print(classification_report(y_test, baseline_pred))

print("\nLogistic Regression Predictions:")
print(classification_report(y_test, mod.predict(X_test)))

Baseline (Random) Accuracy: 0.5077829196466134

Baseline Classification Report:
              precision    recall  f1-score   support

       False       0.51      0.50      0.50      1183
        True       0.51      0.51      0.51      1194

    accuracy                           0.51      2377
   macro avg       0.51      0.51      0.51      2377
weighted avg       0.51      0.51      0.51      2377


Logistic Regression Predictions:
              precision    recall  f1-score   support

       False       0.58      0.76      0.66      1183
        True       0.65      0.44      0.53      1194

    accuracy                           0.60      2377
   macro avg       0.61      0.60      0.59      2377
weighted avg       0.61      0.60      0.59      2377



In [33]:
TP = [a and b for (a,b) in zip(predictions,y)]
TN = [not a and not b for (a,b) in zip(predictions,y)]
FP = [a and not b for (a,b) in zip(predictions,y)]
FN = [not a and b for (a,b) in zip(predictions,y)]

In [34]:
TP = sum(TP)
TN = sum(TN)
FP = sum(FP)
FN = sum(FN)

In [35]:
BER = 0.5 * (FP / (TN + FP) + FN / (FN + TP))


In [36]:
BER

0.510304805485507

### n-grams

In [37]:
sentence = "Dark red color, light beige foam"
unigrams = sentence.split()
bigrams = list(zip(unigrams[:-1], unigrams[1:]))
trigrams = list(zip(unigrams[:-2], unigrams[1:-1], unigrams[2:]))

In [38]:
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
stop_words = set(stopwords.words('english'))
for d in balanced_merged_data:
    review_text = d.get('review_text', '').lower()  # Default to empty string if key is missing

    # Remove punctuation
    r = ''.join([c for c in review_text if c not in punctuation])

    # Tokenize and filter stopwords
    ws = [w for w in r.split() if w not in stop_words]
    ws2 = [' '.join(x) for x in list(zip(ws[:-1],ws[1:]))]
    ws3 = [' '.join(x) for x in list(zip(ws[:-2],ws[1:-1],ws[2:]))]
    ws4 = [' '.join(x) for x in list(zip(ws[:-3],ws[1:-2],ws[2:-1],ws[3:]))]
    ws5 = [' '.join(x) for x in list(zip(ws[:-4],ws[1:-3],ws[2:-2],ws[3:-1],ws[4:]))]
    for w in ws + ws2 + ws3 + ws4 + ws5:
        wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

In [39]:
len(counts)

892203

In [40]:
words = [x[1] for x in counts[:2000]]

In [41]:
wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

In [42]:
def feature(datum):
    # Initialize feature vector with zeros
    feat = [0] * len(words)

    # Safely access and preprocess 'review_text'
    review_text = datum.get('review_text', '').lower()
    r = ''.join([c for c in review_text if c not in punctuation])

    # Tokenize and filter out stopwords
    ws = [w for w in r.split() if w not in stop_words]

    # Generate n-grams (1-grams to 5-grams)
    n_grams = []
    for n in range(1, 6):  # 1-grams to 5-grams
        n_grams += [' '.join(ws[i:i + n]) for i in range(len(ws) - n + 1)]

    # Populate feature vector based on n-grams in 'words'
    for w in n_grams:
        if w in words:
            feat[wordId[w]] += 1

    # Add offset for bias term
    feat.append(1)
    return feat

In [43]:
X = [feature(d) for d in balanced_merged_data]
y = [d['recommend'] for d in balanced_merged_data]

In [44]:
# Convert to numpy arrays
X = np.array(X)
y = np.array(y)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,shuffle=True)

In [45]:
mod = LogisticRegression(C=1)
mod.fit(X_train, y_train)

predictions = mod.predict(X_test) # Binary vector of predictions
correct = predictions == y_test
print(sum(correct) / len(correct))

0.779554059739167


### tf - idf + classifier 

### maybe jacard or cos sim

### data processing:
### grab t

In [46]:
# Count unique users
unique_users = len(set(entry['user_id'] for entry in balanced_merged_data))
print("Number of Unique Users:", unique_users)

# Count unique items
unique_items = len(set(entry['item_id'] for entry in balanced_merged_data))
print("Number of Unique Items:", unique_items)

# Count all items 


Number of Unique Users: 6927
Number of Unique Items: 1777


In [47]:
balanced_merged_data[0]

{'user_id': 'Kitakats',
 'item_id': '244850',
 'review_text': 'Annoying memory leaks, whichs causes long long lag spikes less than 1 fps which last for a few minutes at a time.',
 'recommend': False,
 'playtime_forever': 325,
 'playtime_2weeks': 0}

In [48]:
balanced_merged_data_10 = [
    {**entry, 'recommend': 1 if entry['recommend'] else 0} 
    for entry in balanced_merged_data
]

In [220]:
len(balanced_merged_data_10)

9506

In [490]:
merged_data_10 = [
    {**entry, 'recommend': 1 if entry['recommend'] else 0} 
    for entry in merged_data
]

In [492]:
merged_data_10 = merged_data_10[:20000]

In [493]:
len(merged_data_10)

20000

In [538]:
recommendTrain = merged_data_10[:10000]
recommendValid = merged_data_10[10000:]
recommendPerUser = defaultdict(list)
recommendPerItem = defaultdict(list)
for entry in balanced_merged_data_10:
    u = entry['user_id']
    i = entry['item_id']
    r = entry['recommend']  # Now already 0/1
    
    recommendPerUser[u].append((i,r))
    recommendPerItem[i].append((u,r))

In [362]:

# Create a dictionary to track recommendations per item
item_recommendations = defaultdict(lambda: {'total': 0, 'positive': 0})

# Iterate through the balanced_merged_data
for entry in recommendTrain:
    item_id = entry['item_id']
    recommend = entry['recommend']
    
    # Increment total entries for this item
    item_recommendations[item_id]['total'] += 1
    
    # Increment positive recommendations if recommend is 1
    if recommend == 1:
        item_recommendations[item_id]['positive'] += 1

# Find items with 70% or more positive recommendations
highly_recommended_items = [
    item_id for item_id, data in item_recommendations.items() 
    if data['total'] > 0 and (data['positive'] / data['total']) >= 0.70
]

print("Items with 70% or more positive recommendations:", highly_recommended_items)
print("Total number of such items:", len(highly_recommended_items))

Items with 70% or more positive recommendations: ['49520', '211820', '239140', '1250', '239820', '206440', '230410', '4000', '8870', '220', '265890', '22600', '264710', '441830', '274940', '427250', '223750', '55230', '236730', '214560', '285920', '204360', '300', '273390', '210770', '203160', '42700', '238460', '319630', '374570', '213670', '40800', '400', '353560', '105600', '550', '57300', '8930', '212680', '10090', '449540', '268750', '6060', '233450', '300040', '55040', '212180', '245170', '224260', '200260', '332800', '238320', '10680', '244210', '298110', '330830', '285900', '265630', '335670', '2280', '230230', '231430', '237930', '219740', '264200', '250900', '35450', '252950', '322170', '220200', '203770', '113200', '70400', '4560', '42910', '248350', '7670', '13600', '207140', '200510', '48700', '266510', '305620', '8190', '10', '268500', '302510', '298520', '264380', '4760', '200170', '250320', '266110', '620', '205100', '12750', '280220', '383980', '319740', '354430', '240

In [539]:
usersPerItem = defaultdict(set) # Maps an item to the users who rated it
itemsPerUser = defaultdict(set) # Maps a user to the items that they rated
itemNames = {}
ratingDict = {} # To retrieve a rating for a specific user/item pair

for d in recommendTrain:
    user,item = d['user_id'], d['item_id']
    usersPerItem[item].add(user)
    itemsPerUser[user].add(item)
    ratingDict[(user,item)] = d['recommend']

KeyboardInterrupt: 

Exception ignored in: 'zmq.backend.cython._zmq.Frame.__del__'
Traceback (most recent call last):
  File "_zmq.py", line 160, in zmq.backend.cython._zmq._check_rc
KeyboardInterrupt: 


In [528]:
###Version where only recommended things go into dictionaries 
usersPerItem = defaultdict(set) # Maps an item to the users who recommended it
itemsPerUser = defaultdict(set) # Maps a user to the items they recommended
itemNames = {}
ratingDict = {} # To retrieve a rating for a specific user/item pair

for d in recommendTrain:
    user, item = d['user_id'], d['item_id']
    
    # Only add to sets if recommendation is 1
    if d['recommend'] == 1:
        usersPerItem[item].add(user)
        itemsPerUser[user].add(item)
    
    # Keep ratingDict for all entries
    ratingDict[(user,item)] = d['recommend']

In [536]:
reviewsPerUser = defaultdict(list)
reviewsPerItem = defaultdict(list)
for d in recommendTrain:
    user,item = d['user_id'], d['item_id']
    reviewsPerUser[user].append(d)
    reviewsPerItem[item].append(d)

In [534]:
##same thing, append only if recommend is 1 (True)
#reviewsPerUser = defaultdict(list)
#reviewsPerItem = defaultdict(list)

for d in recommendTrain:
    user, item = d['user_id'], d['item_id']
    
    # Only append the review if the recommendation is 1
    if d['recommend'] == 1:
        reviewsPerUser[user].append(d)
        reviewsPerItem[item].append(d)

In [448]:
ratingMean = sum([d['recommend'] for d in recommendTrain]) / len(recommendTrain) #0.5 obviously, change later to heuristic

In [383]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom > 0:
        return numer/denom
    return 0

In [384]:
def CosineSet(s1, s2):
    # Not a proper implementation, operates on sets so correct for interactions only
    numer = len(s1.intersection(s2))
    denom = math.sqrt(len(s1)) * math.sqrt(len(s2))
    if denom == 0:
        return 0
    return numer / denom

In [530]:
def predictRating1(user,item):
    ratings = []
    similarities = []
    maxSim = 0
    for d in reviewsPerUser[user]:
        i2 = d['item_id']
        if i2 == item: continue
        ratings.append(d['recommend'])
        value = CosineSet(usersPerItem[item],usersPerItem[i2])
        maxSim = max(value, maxSim)
        #similarities.append(CosineSet(usersPerItem[item],usersPerItem[i2]))

    #if (sum(similarities) > 0):
        #weightedRatings = [(x*y) for x,y in zip(ratings,similarities)]
        #return 1 if sum(weightedRatings) / sum(similarities) >= 0.9 else 0
    if maxSim > 0.005:
        return 1
    else:
        return 0#round(ratingMean)

In [451]:
def calculate_accuracy(predictions, labels):
    # Calculate the number of correct predictions
    correct_predictions = sum(1 for pred, label in zip(predictions, labels) if pred == label)
    
    # Calculate accuracy as the percentage of correct predictions
    accuracy = correct_predictions / len(labels)
    
    return accuracy

In [531]:
def predictRating2(user, item):
    ratings = []
    similarities = []
    maxSim = 0
    for d in reviewsPerItem[item]:
        u2 = d['user_id']
        if u2 == user: continue
        ratings.append(d['recommend'])
        value = CosineSet(itemsPerUser[user], itemsPerUser[u2])
        maxSim = max(maxSim,value)
        
    if maxSim > 0.005:
        return 1
        #weightedRatings = [(x*y) for x,y in zip(ratings, similarities)]
        # Round to 0 or 1 for binary classification
        
        #return 1 if sum(weightedRatings) / sum(similarities) >= 0.001 else 0
    else:
        return predictRating1(user,item)

In [231]:
u,i = balanced_merged_data_10[4]['user_id'], balanced_merged_data_10[4]['item_id']
print(predictRating1(u, i))
print(balanced_merged_data_10[4]['user_id'])
print(balanced_merged_data_10[4]['item_id'])
print(balanced_merged_data_10[4])

0
76561198079355713
49520
{'user_id': '76561198079355713', 'item_id': '49520', 'review_text': 'Loads of fun. Very re-playable. Must have', 'recommend': 1, 'playtime_forever': 18100, 'playtime_2weeks': 0}


In [295]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

In [388]:
alwaysPredictMean = [ratingMean for d in recommendValid]

In [389]:
simPredictions = [predictRating2(d['user_id'], d['item_id']) for d in recommendValid]

In [402]:
labels = [d['recommend'] for d in recommendValid]

In [403]:
MSE(alwaysPredictMean, labels)

0.5037727474478473

In [404]:
MSE(simPredictions, labels)

0.5022192632046161

In [537]:
# Always predict mean baseline (rounded to 0 or 1)
#alwaysPredictMean = [1 if ratingMean >= 0.5 else 0 for d in recommendValid]
alwaysPredictMean = [random.randint(0, 1) for d in recommendValid]

# Get predictions using the modified predictRating2
simPredictions = [predictRating2(d['user_id'], d['item_id']) for d in recommendValid]

# Get true labels
labels = [d['recommend'] for d in recommendValid]

# Calculate accuracies
mean_baseline_accuracy = calculate_accuracy(alwaysPredictMean, labels)
similarity_based_accuracy = calculate_accuracy(simPredictions, labels)

print("Mean Baseline Accuracy:", mean_baseline_accuracy)
print("Similarity-based Prediction Accuracy:", similarity_based_accuracy)

Mean Baseline Accuracy: 0.49416666666666664
Similarity-based Prediction Accuracy: 0.106


### latent factors (svs or matrix factorization)

In [51]:
len(cleaned_item_metadata_further)

1235

In [52]:
len(balanced_merged_data) 

9506

In [53]:
# Step 1: Create a set of 'id' values from cleaned_item_metadata_further for fast lookup
item_metadata_ids = {entry['id'] for entry in cleaned_item_metadata_further}

# Step 2: Create a new list to hold the combined entries
combined_data = []

# Step 3: Iterate through balanced_merged_data and combine with metadata
for entry in balanced_merged_data:
    item_id = entry['item_id']
    
    # Step 4: If the item_id exists in cleaned_item_metadata_further, combine data
    if item_id in item_metadata_ids:
        # Find the corresponding metadata entry in cleaned_item_metadata_further
        metadata_entry = next(item for item in cleaned_item_metadata_further if item['id'] == item_id)
        
        # Combine the metadata with the entry from balanced_merged_data
        combined_entry = {
            **entry,  # Include all fields from balanced_merged_data entry
            'publisher': metadata_entry.get('publisher'),
            'genres': metadata_entry.get('genres'),
            'tags': metadata_entry.get('tags'),
            'price': metadata_entry.get('price')
        }
        
        # Add the combined entry to the new list
        combined_data.append(combined_entry)

# Step 5: Optionally, print the number of combined entries
print(f"Combined data contains {len(combined_data)} entries.")

Combined data contains 6871 entries.


In [54]:
combined_data[0]

{'user_id': '76561198080466065',
 'item_id': '273350',
 'review_text': 'besides planet scanner its not too bad imo',
 'recommend': True,
 'playtime_forever': 605,
 'playtime_2weeks': 0,
 'publisher': '2K',
 'genres': ['Action', 'Free to Play'],
 'tags': ['Action',
  'Multiplayer',
  'Co-op',
  'FPS',
  'Online Co-Op',
  'Shooter',
  'Team-Based',
  'Survival',
  'Sci-fi',
  'Aliens',
  'PvP',
  'First-Person',
  'Class-Based',
  'Strategy',
  'Atmospheric',
  'Third Person',
  'Tactical',
  'Singleplayer',
  'Stealth',
  'Casual'],
 'price': 'Free to Play'}

In [55]:
# Step 1: Create a set of (user_id, item_id) pairs from balanced_merged_data
balanced_merged_data_pairs = {
    (entry['user_id'], entry['item_id']) for entry in balanced_merged_data
}

# Step 2: Check if each (user_id, item_id) in combined_data exists in balanced_merged_data_pairs
for entry in combined_data:
    user_id = entry['user_id']
    item_id = entry['item_id']
    
    # If the pair (user_id, item_id) is not found in balanced_merged_data_pairs
    if (user_id, item_id) not in balanced_merged_data_pairs:
        print(f"Mismatch found: user_id={user_id}, item_id={item_id}")
        # You can also raise an error here if needed, for example:
        # raise ValueError(f"Pair not found: user_id={user_id}, item_id={item_id}")

# Step 3: If no mismatches are found, print a confirmation message
print("All pairs in combined_data are present in balanced_merged_data.")

All pairs in combined_data are present in balanced_merged_data.


In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Step 1: Combine genres and tags into a single string for each game
combined_data_with_text = []
for review in combined_data:
    genres_text = " ".join(review['genres']) if review['genres'] else ""
    tags_text = " ".join(review['tags']) if review['tags'] else ""
    combined_data_with_text.append(genres_text + " " + tags_text)


In [57]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(combined_data_with_text)

In [58]:
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

In [59]:
print(tfidf_df.head())

   1980s  1990        2d   3d  40k   4x   5d  abstract  access    action  ...  \
0    0.0   0.0  0.000000  0.0  0.0  0.0  0.0       0.0     0.0  0.160832  ...   
1    0.0   0.0  0.000000  0.0  0.0  0.0  0.0       0.0     0.0  0.185002  ...   
2    0.0   0.0  0.000000  0.0  0.0  0.0  0.0       0.0     0.0  0.185002  ...   
3    0.0   0.0  0.000000  0.0  0.0  0.0  0.0       0.0     0.0  0.216131  ...   
4    0.0   0.0  0.210677  0.0  0.0  0.0  0.0       0.0     0.0  0.168524  ...   

   web  werewolves  western  with  word  workshop     world  wrestling  your  \
0  0.0         0.0      0.0   0.0   0.0       0.0  0.000000        0.0   0.0   
1  0.0         0.0      0.0   0.0   0.0       0.0  0.000000        0.0   0.0   
2  0.0         0.0      0.0   0.0   0.0       0.0  0.000000        0.0   0.0   
3  0.0         0.0      0.0   0.0   0.0       0.0  0.115988        0.0   0.0   
4  0.0         0.0      0.0   0.0   0.0       0.0  0.135659        0.0   0.0   

   zombies  
0      0.0  
1     

### one hot encodings of genres and tags

In [60]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from scipy.sparse import hstack, csr_matrix

In [61]:
def prepare_features(combined_data, max_features=1000):
    # Create a DataFrame
    df = pd.DataFrame(combined_data)
    
    # Quick price conversion
    df['price'] = pd.to_numeric(df['price'].replace('Free to Play', 0), errors='coerce').fillna(0)
    
    # Faster list handling
    df['genres'] = df['genres'].apply(lambda x: x if isinstance(x, list) else [])
    df['tags'] = df['tags'].apply(lambda x: x if isinstance(x, list) else [])
    
    # More efficient MultiLabelBinarizer
    mlb_genres = MultiLabelBinarizer(sparse_output=False)
    mlb_tags = MultiLabelBinarizer(sparse_output=True)
    
    # Sparse one-hot encoding
    genres_encoded = mlb_genres.fit_transform(df['genres'])
    tags_encoded = mlb_tags.fit_transform(df['tags'])
    
    # Sparse feature matrix construction
    #price_col = csr_matrix(df['price'].values.reshape(-1, 1))
    #playtime_col = csr_matrix(df['playtime_forever'].values.reshape(-1, 1))
    
    # Combine features sparsely
    X = hstack([
        #price_col, 
        #playtime_col, 
        genres_encoded[:, :max_features], 
        tags_encoded[:, :max_features]
    ])
    
    # Prepare target
    y = df['recommend'].astype(int)
    
    # Collect feature names for later use
    feature_names = ['price', 'playtime_forever'] + list(mlb_genres.classes_[:max_features]) + list(mlb_tags.classes_[:max_features])
    
    return X, y, feature_names

def train_and_evaluate(X, y, feature_names):
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
    
    # Train with fewer iterations and stronger regularization
    model = LogisticRegression(max_iter=1200, C=1)
    model.fit(X_train, y_train)
    
    # Evaluate
    y_pred = model.predict(X_test)
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    
    # Show important features (sorted by absolute value of coefficients)
    importance = np.abs(model.coef_).flatten()
    sorted_idx = np.argsort(importance)[::-1]  # Sort in descending order
    
    print("\nTop important features:")
    for idx in sorted_idx[:40]:  # Show top 10 important features
        print(f"{feature_names[idx]}: {importance[idx]:.4f}")
    
    return model

# Main execution
X, y, feature_names = prepare_features(combined_data)
#convert to 1/0 replacing recomment true/false with 1/0
model = train_and_evaluate(X, y, feature_names)

Accuracy: 0.6746216530849826

Classification Report:
              precision    recall  f1-score   support

           0       0.67      0.63      0.65       816
           1       0.68      0.72      0.70       902

    accuracy                           0.67      1718
   macro avg       0.67      0.67      0.67      1718
weighted avg       0.67      0.67      0.67      1718


Top important features:
Satire: 1.8422
Atmospheric: 1.5336
Metroidvania: 1.4149
Intentionally Awkward Controls: 1.2884
3D Vision: 1.2822
Video Production: 1.2460
Quick-Time Events: 1.2193
First-Person: 1.2099
Gambling: 1.1534
Difficult: 1.1389
Turn-Based Strategy: 1.1050
Dark: 1.1038
Alternate History: 1.0785
Typing: 1.0642
Sports: 1.0642
2D Fighter: 1.0433
Strategy: 1.0188
Hacking: 1.0133
Villain Protagonist: 1.0112
Score Attack: 1.0075
Funny: 1.0056
Sailing: 1.0022
Kickstarter: 0.9453
Science: 0.9424
Casual: 0.9399
Historical: 0.9390
Turn-Based: 0.9294
3D Platformer: 0.9009
Aliens: 0.8898
Casual: 0.8866
Wrestl

In [62]:
def CosineSet(s1, s2):
    # Not a proper implementation, operates on sets so correct for interactions only
    numer = len(s1.intersection(s2))
    denom = math.sqrt(len(s1)) * math.sqrt(len(s2))
    if denom == 0:
        return 0
    return numer / denom

In [63]:
len(balanced_merged_data)

9506